This notebook is for data extraction from PDFs

In [4]:
# Install PyPDF2
!pip install PyPDF2

import PyPDF2
import os
import time
import pandas as pd
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/GOT tSNE')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/GOT tSNE'

In [0]:
import time 

start = time.time()
time.sleep(5)
print("Total time taken is {} seconds.".format(round((time.time() - start), 2)))

Total time taken is 5.01 seconds.


In [0]:
def extractPdfText(filePath=''):
    '''
    Function will extract and return the pdf file text content.
    '''
    
    fileObject = open(filePath, 'rb') # Open the pdf file in read binary mode.

    pdfFileReader = PyPDF2.PdfFileReader(fileObject) # Create a pdf reader .

    totalPages = pdfFileReader.numPages # Get total pdf page number.
    
    print('This pdf file contains totally ' + str(totalPages) + ' pages.') # Print pdf total page number.
    
    textContent = ""

    for currentPage in tqdm(range(totalPages)):
        pdfPage = pdfFileReader.getPage(currentPage)
        try:
          text = pdfPage.extractText()
#           print("Extrcted text is", text)
          textContent += text
          if len(text) > 0:
            print("page {} working".format(currentPage))
        except:
          print("page {} not working". format(currentPage))
          continue
          
    return textContent

In [0]:
# # del_ = extractPdfText(os.listdir(os.getcwd())[0])
del_ = extractPdfText("data/A Game Of Thrones - George RR Martin.pdf")
# # length of extract string is 1444222

In [6]:
os.listdir(os.getcwd() + "/data/PDFs")

['A Game Of Thrones - George RR Martin.pdf',
 'A Clash of Kings - George RR Martin.pdf',
 'A Storm of Swords - George RR Martin.pdf',
 'A Feast For Crows - George RR Martin.pdf',
 'A Dance With Dragons - George R R Martin.pdf']

# Code to extract text from one book 

In [0]:
def extractPdfText(filePath=''):
    '''
    Function will extract and return the pdf file text content.
    '''
    
    fileObject = open(filePath, 'rb') # Open the pdf file in read binary mode.

    pdfFileReader = PyPDF2.PdfFileReader(fileObject) # Create a pdf reader .

    totalPages = pdfFileReader.numPages # Get total pdf page number.
    
    print('This pdf file contains totally ' + str(totalPages) + ' pages.') # Print pdf total page number.
    
    df = pd.DataFrame()
    for currentPage in range(totalPages):
        pdfPage = pdfFileReader.getPage(currentPage)
        try:
          text = pdfPage.extractText()
          df.loc[currentPage, 'text'] = text
          
#           if len(text) > 0:
#             print("page {} working".format(currentPage))
        except:
          print("page {} not working". format(currentPage))
          pass
          
    return df

In [18]:
df = pd.DataFrame()
start = time.time()

filename =  'A Dance With Dragons - George R R Martin.pdf'
book5_df = extractPdfText(os.getcwd() + "/data/PDFs/" + filename)

end = time.time()

This pdf file contains totally 916 pages.
page 1 not working


In [19]:
len(book5_df)

915

In [21]:
book5_df.tail()

,text
911,"\n \n \nNew York, NY 10022\n \n \n \n \nhttp:..."
912,\n \n \n
913,
914,\n \n \n \n
915,\n \n


In [52]:
documents = [gensim.utils.simple_preprocess(row) for row in book5_df['text']]

type(documents)

model = gensim.models.Word2Vec(
        documents,
        size=150,
        window=10,
        min_count=2,
        workers=10)
model.train(documents, total_examples=len(documents), epochs=10)

(1783981, 2411040)

In [57]:
# model.wv.most_similar('jon')
len(model.wv.vocab)

8687

In [51]:
documents

['tyrion',
 'davos',
 'daenerys',
 'jon',
 'tyrion',
 'davos',
 'reek',
 'jon',
 'tyrion',
 'daenerys',
 'the',
 'lost',
 'lord',
 'the',
 'windblown',
 'the',
 'wayward',
 'bride']

In [0]:
r = [['mayor', 'new_york'],
     ['machin', 'learn', 'us'],
     ['new_york', 'mayor', 'present'],
     ['mayor', 'new_york'],
     ['machin', 'learn', 'us'],
     ['new_york', 'mayor', 'present']]

In [43]:
model = gensim.models.Word2Vec(
        r,
        size=150,
        window=10,
        min_count=2,
        workers=10)
model.train(documents, total_examples=len(documents), epochs=10)

(0, 10360520)

# Extract text from EPUBs

In [0]:
!pip install ebooklib

In [0]:
os.listdir(os.getcwd() + "/data/EPUBs")

['A Dance With Dragons - George R R Martin.epub',
 'A Feast For Crows - George RR Martin.epub',
 'A Storm of Swords - George RR Martin.epub',
 'A Clash of Kings - George RR Martin.epub',
 'A Game Of Thrones - George RR Martin.epub']

In [0]:
import ebooklib
from ebooklib import epub

book = epub.read_epub('data/EPUBs/A Game Of Thrones - George RR Martin.epub')

book.get_metadata('DC', 'creator')
# pass namespace and name

[('George R. R. Martin',
  {'{http://www.idpf.org/2007/opf}file-as': 'Martin, George R. R.',
   '{http://www.idpf.org/2007/opf}role': 'aut'})]

In [0]:
# doc = book.get_items_of_type(ebooklib.ITEM_DOCUMENT)
for item in book.get_items():
  if item.get_type() == 9:
    print(item.get_body_content())
    
# this code can be used if the item.get_body_content() is converted to string
# and concatenated and cleaned later on. 

for doc in book.get_items():
    doc_content += str(doc.content)
    break

# Code to load from Siraj's 5 txt files

Takes 10 minutes to execute - large index value problem...change code to get each df and append it to master - so that index never grows that large. 

Output saved to `data/allbooksSiraj.csv`

In [0]:
start = time.time()

df = pd.DataFrame()
for txt_file in os.listdir(os.getcwd() + "/data/txt_files"):
  i = len(df)
  with open("data/txt_files/" + txt_file, 'r') as f:
    for idx, row in enumerate(f):
      df.loc[idx + i, 'text'] = row

end = time.time()

if (end - start) <= 60:
  print("Total time taken is {} seconds.".format(round(end - start, 2)))
else:
  print("Total time taken is {} minutes.".format(round((end - start)/60, 2)))